In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day12.jpeg
/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/ryct.2020200034.fig5-day4.jpeg
/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5.jpeg
/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/ryct.2020200034.fig5-day0.jpeg
/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/SARS-10.1148rg.242035193-g04mr34g09a-Fig9a-day17.jpeg
/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/ryct.2020200034.fig2.jpeg
/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/streptococcus-pneumoniae-pneumonia-temporal-evolution-1-day3.jpg
/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PN

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_ds = image_dataset_from_directory(
    "/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train",
    image_size=(224, 224),
    color_mode="grayscale",
    batch_size=32
)

test_ds = image_dataset_from_directory(
    "/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test",
    image_size=(224, 224),
    color_mode="grayscale",
    batch_size=32
)




Found 148 files belonging to 2 classes.
Found 40 files belonging to 2 classes.


In [16]:
import keras
from keras import layers

In [17]:
inputs = keras.Input(shape=(224, 224, 1))

x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(2, activation='softmax')(x)

model = keras.Model(inputs, outputs)


In [18]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(train_ds, validation_data=test_ds, epochs=15)

test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.2f}")


Epoch 1/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.4461 - loss: 20.0402 - val_accuracy: 0.5000 - val_loss: 2.6310
Epoch 2/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5037 - loss: 2.2068 - val_accuracy: 0.5000 - val_loss: 0.7021
Epoch 3/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.4834 - loss: 0.7784 - val_accuracy: 0.8750 - val_loss: 0.6441
Epoch 4/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5147 - loss: 0.6875 - val_accuracy: 0.5000 - val_loss: 0.6589
Epoch 5/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.6552 - loss: 0.6206 - val_accuracy: 0.9000 - val_loss: 0.4914
Epoch 6/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7430 - loss: 0.5496 - val_accuracy: 0.5500 - val_loss: 0.5240
Epoch 7/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.6584 - loss: 0.6001 - val_accuracy: 0.5000 - val_loss: 0.6073
Epoch 8/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.6978 - loss: 0.5436 - val_accuracy: 1.0000 - val_loss: 0.2865
Epoch 9

### the above one took high amount of time

### Now using the pretrained model ResNet50

In [19]:
from tensorflow import keras
from tensorflow.keras import layers

base_model = keras.applications.ResNet50(
    include_top=False,             
    weights='/kaggle/input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',            # load pretrained weights
    input_shape=(224, 224, 3)
)


In [21]:
base_model.trainable = False


In [24]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(2, activation='softmax')(x)

model = keras.Model(inputs, outputs)
train_ds = train_ds.map(lambda x, y: (tf.image.grayscale_to_rgb(x), y))
test_ds = test_ds.map(lambda x, y: (tf.image.grayscale_to_rgb(x), y))

In [25]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5
)


Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.4705 - loss: 1.8168 - val_accuracy: 0.5250 - val_loss: 1.2802
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.4467 - loss: 1.7842 - val_accuracy: 0.5500 - val_loss: 1.0016
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.3952 - loss: 1.4830 - val_accuracy: 0.6000 - val_loss: 0.7999
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.4383 - loss: 1.3043 - val_accuracy: 0.6750 - val_loss: 0.6755
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.4818 - loss: 1.0569 - val_accuracy: 0.7500 - val_loss: 0.6015


In [26]:
base_model.trainable = True

# Freeze all layers except the top 20%
fine_tune_at = int(len(base_model.layers) * 0.8)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5), 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5
)


Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 44s 5s/step - accuracy: 0.4913 - loss: 1.5305 - val_accuracy: 0.7250 - val_loss: 0.5045
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.5612 - loss: 0.9294 - val_accuracy: 0.7500 - val_loss: 0.4365
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.6071 - loss: 0.6942 - val_accuracy: 0.8000 - val_loss: 0.3786
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.8028 - loss: 0.3630 - val_accuracy: 0.8750 - val_loss: 0.3242
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.9540 - loss: 0.1724 - val_accuracy: 0.9000 - val_loss: 0.2762


In [27]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc*100:.2f}%")


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 560ms/step - accuracy: 0.9021 - loss: 0.2762
Test accuracy: 90.00%
